In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

response_template = "\n\n### Response:\n"
collator = DataCollatorForCompletionOnlyLM(response_template=response_template, tokenizer=tokenizer, mlm=False)

trainer = SFTTrainer(
    model,
    args=SFTConfig(output_dir="/tmp", per_device_train_batch_size=4),
    train_dataset=dataset,
    data_collator=collator,
)

for batch in trainer.get_train_dataloader():
    print(len(batch))
    break

batch['labels'][batch["labels"] == -100] = 1
inputs = tokenizer.batch_decode(batch["input_ids"])
targets = tokenizer.batch_decode(batch["labels"])
print(len(inputs))
print(inputs[0])
print("*" * 50)
print(targets[0])

Repo card metadata block was not found. Setting CardData to empty.


3
4
</s>### Human: 回答集编程和Prolog有什么联系和区别？### Assistant: 回答集编程（Answer Set Programming，ASP）和Prolog是两种逻辑编程语言，它们之间的联系和区别如下：

联系：
1. ASP和Prolog都使用规则和查询来表示和解决问题。
2. ASP和Prolog都支持递归和模式匹配。
3. ASP和Prolog都可以被用来表示和解决复杂的逻辑和知识表达式。

区别：
1. ASP的语法更接近于一种自然语言，而Prolog的语法更接近于一种函数式编程语言。
2. ASP和Prolog的语义不同。ASP使用基于稳定模型（stable model）的语义来定义规则的解释，而Prolog使用基于霍恩子句（Horn clause）的语义来定义规则的解释。
3. ASP和Prolog的应用领域不同。ASP通常用于复杂知识的表示和解决，如自然语言处理、图形学等领域，而Prolog通常用于人工智能、专家系统等领域。### Human: 看来prolog也是挺强大的。为什么它现在不火呢？请试着给出几个原因。### Assistant: Prolog在当今编程语言市场上的影响力不如一些其他语言，可能是由于以下几个原因：

1. **学习曲线较陡峭**
Prolog语法和概念与传统的命令式编程语言非常不同，这使得初学者在学习过程中可能会遇到一些挑战。在传统的命令式编程语言中，程序员需要考虑如何控制程序的流程以及如何处理数据。而在Prolog中，程序员需要思考如何描述问题和约束条
**************************************************
<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

/Users/jackyu/Developer/multilingual-alpaca-qlora/.venv/lib/python3.11/site-packages/trl/trainer/utils.py:160: UserWarning: Could not find response key `

### Response:
` in the following instance: </s>### Human: 回答集编程和Prolog有什么联系和区别？### Assistant: 回答集编程（Answer Set Programming，ASP）和Prolog是两种逻辑编程语言，它们之间的联系和区别如下：

联系：
1. ASP和Prolog都使用规则和查询来表示和解决问题。
2. ASP和Prolog都支持递归和模式匹配。
3. ASP和Prolog都可以被用来表示和解决复杂的逻辑和知识表达式。

区别：
1. ASP的语法更接近于一种自然语言，而Prolog的语法更接近于一种函数式编程语言。
2. ASP和Prolog的语义不同。ASP使用基于稳定模型（stable model）的语义来定义规则的解释，而Prolog使用基于霍恩子句（Horn clause）的语义来定义规则的解释。
3. ASP和Prolog的应用领域不同。ASP通常用于复杂知识的表示和解决，如自然语言处理、图形学等领域，而Prolog通常用于人工智能、专家系统等领域。### Human: 看来prolog也是挺强大的。为什么它现在不火呢？请试着给出几个原因。### Assistant: Prolog在当今编程语言市场上的影响力不如一些其他语言，可能是由于以下几个原因：

1. **学习曲线较陡峭**
Prolog语法和概念与传统的命令式编程语言非常不同，这使得初学者在学习过程中可能会遇到一些挑战。在传统的命令式编程语言中，程序员需要考虑如何控制程序的流程以及如何处理数据。而在Prolog中，程序员需要思考如何描述问题和约束条 This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_seq_length`.
  warnin

In [33]:
dataset

Dataset({
    features: ['text'],
    num_rows: 9846
})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from utils.prompter import Prompter

dataset = load_dataset("yahma/alpaca-cleaned", split="train")
prompter = Prompter()
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")
tokenizer.pad_token = tokenizer.eos_token

def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],
    )

    full_prompt_tokens = tokenizer.encode(full_prompt, return_tensors="pt")
    output_tokens = tokenizer.encode(data_point["output"], return_tensors="pt")

    response_start_index = full_prompt_tokens.shape[1] - output_tokens.shape[1] 

    return {
        "text": full_prompt, 
        # "response_start_index": response_start_index
    }

dataset = dataset.map(generate_and_tokenize_prompt)
dataset = dataset.remove_columns(["instruction", "input", "output"])


In [5]:
instruction_template = "\n\n### Instruction:\n"
response_template = "\n\n### Response:\n"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)

trainer = SFTTrainer(
    model,
    args=SFTConfig(output_dir="/tmp", per_device_train_batch_size=4),
    train_dataset=dataset,
    data_collator=collator,
)

for batch in trainer.get_train_dataloader():
    print(len(batch), batch)
    break

batch['labels'][batch["labels"] == -100] = 1
inputs = tokenizer.batch_decode(batch["input_ids"])
targets = tokenizer.batch_decode(batch["labels"])
print("batch_size:", len(inputs))
print(inputs[0])
print("*" * 50)
print(targets[0])

/Users/jackyu/Developer/multilingual-alpaca-qlora/.venv/lib/python3.11/site-packages/trl/trainer/utils.py:134: UserWarning: The pad_token_id and eos_token_id values of this tokenizer are identical. If you are planning for multi-turn training, it can result in the model continuously generating questions and answers without eos token. To avoid this, set the pad_token_id to a different value.
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'config'

In [31]:
tokenizer.pad_token = tokenizer.eos_token
texts = ["How are you?", "What is the capital of France?"]

# [tokenizer.apply_chat_template([text]) for text in texts]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs, output_hidden_states=True)

In [32]:
# decode logits to text
decoded_outputs = tokenizer.batch_decode(outputs.logits.argmax(2), skip_special_tokens=True)
decoded_outputs

['The to the doing I\xa0’t\xa0', 'The is the difference of the?\n Paris']

In [41]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features

In [1]:
len(outputs.hidden_states)

NameError: name 'outputs' is not defined

In [ ]:
class LanguageContrastiveSFTTrainer(SFTTrainer):
    def compute_contrastive_loss(self, outputs):
        first_layer_hidden_states = outputs.hidden_states[1]
        hidden_states = torch.stack(hidden_states, dim=1)
        hidden_states = hidden_states[:, -1]
    
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        print("Computing SFT loss")
        nll_loss, outputs = super().compute_loss(model, inputs, return_outputs=True, num_items_in_batch=num_items_in_batch)
        print("Computing contrastive loss")
        contrastive_loss = self.compute_contrastive_loss(outputs)
        alpha = self.args.alpha
        loss = (1 - alpha) * nll_loss + alpha * contrastive_loss
        return (loss, outputs) if return_outputs else loss